In [70]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
import pandas as pd
from IPython.display import Image, display, Math
from PIL import Image as ImagePIL

First of all, we derive the DH-table to then get the roation matrices:
| **Joint (i)** | **θᵢ**   | **dᵢ**   | **aᵢ₋₁** | **αᵢ₋₁** |
|---------------|----------|----------|----------|----------|
| 1             | $\theta_1$       | 0        | 0        | 0        |
| 2             | $\theta_2$       | 0        | $L3$| 0        |
| 3             | $\theta_3$       | 0        | $L_1$| 0        |
| 4             | $\theta_4$       | 0        | $L2$| 0        |
| EE             | 0       | 0        | $5.221$| 0        |

In [23]:
L1 = 2.194
L2 = 2.186
L4 = 6.196
M3 = 0.1
M4 = 0.1

In [24]:
# Function for deriving the Transformation Matrix
def transformation_matrix(theta, d, a, alpha):
    return sp.Matrix([
        [sp.cos(theta), -sp.sin(theta), 0, a],
        [sp.sin(theta) * sp.cos(alpha), sp.cos(theta) * sp.cos(alpha), -sp.sin(alpha), -sp.sin(alpha) * d],
        [sp.sin(theta) * sp.sin(alpha), sp.cos(theta) * sp.sin(alpha), sp.cos(alpha), sp.cos(alpha) * d],
        [0, 0, 0, 1]
    ])

In [25]:
# Create symbolic variables of the lengths and thetas with the thetas are a function of the time.
t= sp.symbols("t")
theta1 = sp.Function('theta_1')(t)
theta2 = sp.Function('theta_2')(t)
theta3 = sp.Function('theta_3')(t)
theta4 = sp.Function('theta_4')(t)
theta1_dot = sp.Derivative(theta1, t)
theta2_dot = sp.Derivative(theta2, t)
theta3_dot = sp.Derivative(theta3, t)
theta4_dot = sp.Derivative(theta4, t)
theta1_ddot = sp.Derivative(theta1, t, t)
theta2_ddot = sp.Derivative(theta2, t, t)
theta3_ddot = sp.Derivative(theta3, t, t)
theta4_ddot = sp.Derivative(theta4, t, t)

In [26]:
T03 = transformation_matrix(theta3, 0, L1, 0)
T34 = transformation_matrix(theta4, 0, L2, 0)
T4EE = transformation_matrix(0, 0, L4, 0)

T04 = T03 * T34
T0EE = T04 * T4EE

R03 = T03[:3, :3]
R34 = T34[:3, :3]


In [27]:
w0 = sp.Matrix([0, 0, 0])
w3 = R03.T * w0 + sp.Matrix([0, 0, theta3_dot])
w4 = R34.T * w3 + sp.Matrix([0, 0, theta4_dot])

In [28]:
# Postion Coordinates from base frame to the corresponding center of mass
P_c3 = (T03 * sp.Matrix([L2/2, 0, 0, 1]))[:3, :]
P_c4 = (T04 * sp.Matrix([L4/2, 0, 0, 1]))[:3, :]

In [29]:
v_c3 = P_c3.diff(t)
v_c4 = P_c4.diff(t)

In [30]:
I_c3 = sp.Matrix([
    [1/12 * M3 * L2**2, 0, 0],
    [0, 1/12 * M3 * L2**2, 0],
    [0, 0, 0]
])

I_c4 = sp.Matrix([
    [1/12 * M4 * L4**2, 0, 0],
    [0, 1/12 * M4 * L4**2, 0],
    [0, 0, 0]
])

In [31]:
k_c3 = 0.5 * M3 * v_c3.T * v_c3 + 0.5 * w3.T * I_c3 * w3
k_c4 = 0.5 * M4 * v_c4.T * v_c4 + 0.5 * w4.T * I_c4 * w4

# Total Kinetic Energy:
K = k_c3 + k_c3

In [32]:
# We assume that the gravity act in negative y-direction of the base frame
g_vec = sp.Matrix([0, -9.81, 0])

# Potential energies:
u_3 = -M3 * g_vec.T * P_c3
u_4 = -M4 * g_vec.T * P_c4

# Total Potential Energy:
U = u_3 + u_4

In [33]:
# Potential Energy derivates
U_diff_theta1 = U.diff(theta1)
U_diff_theta3 = U.diff(theta3)

# Kinetic Energy derivates
K_diff_theta1 = K.diff(theta1)
K_diff_theta3 = K.diff(theta3)

# Kinetic Energy time derivates
# First differentiate wrt. the derivates of the thetas: theta_dots
K_diff_theta1_dot = K.diff(theta1_dot)
K_diff_theta3_dot = K.diff(theta3_dot)
# Now differentiate wrt. time: t
K_diff_theta1_dot_diff_t = K_diff_theta1_dot.diff(t)
K_diff_theta3_dot_diff_t = K_diff_theta3_dot.diff(t)

In [34]:
tau = sp.Matrix([K_diff_theta1_dot_diff_t - K_diff_theta1 + U_diff_theta1
                 , K_diff_theta3_dot_diff_t - K_diff_theta3 + U_diff_theta3])

------------------------------------

| **Joint (i)** | **θᵢ**   | **dᵢ**   | **aᵢ₋₁** | **αᵢ₋₁** |
|---------------|----------|----------|----------|----------|
| EE             | $\theta_{EE}$       | 0        | 0        | 0        |
| 4             | $\theta_4$       | 0        | L4        | 0        |
| 3             | $\theta_3$       | 0        | L2        | 0        |

In [49]:
L1 = 2.194
L2 = 2.186
L4 = 6.196
LCG = 5
Lx, Ly, Lz = 20, 10, 20
M = 3
M2 = 0.1
M4 = 0.1

In [50]:
# Create symbolic variables of the lengths and thetas with the thetas are a function of the time.
t= sp.symbols("t")
thetaEE = sp.Function('theta_EE')(t)
theta3 = sp.Function('theta_3')(t)
theta4 = sp.Function('theta_4')(t)
thetaEE_dot = sp.Derivative(thetaEE, t)
theta3_dot = sp.Derivative(theta3, t)
theta4_dot = sp.Derivative(theta4, t)
thetaEE_ddot = sp.Derivative(thetaEE, t, t)
theta3_ddot = sp.Derivative(theta3, t, t)
theta4_ddot = sp.Derivative(theta4, t, t)

In [51]:
T0EE = transformation_matrix(thetaEE, 0, 0, 0)
TEE4 = transformation_matrix(theta4, 0, L4, 0)
T43 = transformation_matrix(theta3, 0, L2, 0)

T04 = T0EE * TEE4
T03 = T04 * T43

R_0EE = T0EE[:3, :3]
R_EE4 = TEE4[:3, :3]
R_43 = T43[:3, :3]

In [52]:
w_0 = sp.Matrix([0, 0, 0])
w_EE = R_0EE.T * w_0 + sp.Matrix([0, 0, thetaEE_dot])
w_4 = R_EE4.T * w_EE + sp.Matrix([0, 0, theta4_dot])
w_3 = R_43.T * w_4 + sp.Matrix([0, 0, theta3_dot])

In [53]:
P_cEE = (T0EE * sp.Matrix([L4/2, 0, 0, 1]))[:3, :]
P_c4 = (T04 * sp.Matrix([L2/2, 0, 0, 1]))[:3, :]
P_c3 = (T03 * sp.Matrix([LCG, 0, 0, 1]))[:3, :]

In [54]:
v_cEE = P_cEE.diff(t)
v_c4 = P_c4.diff(t)
v_c3 = P_c3.diff(t)

In [55]:
I_cEE = sp.Matrix([
    [1/12 * M4 * L4**2, 0, 0],
    [0, 1/12 * M4 * L4**2, 0],
    [0, 0, 0]
])

I_c4 = sp.Matrix([
    [1/12 * M2 * L2**2, 0, 0],
    [0, 1/12 * M2 * L2**2, 0],
    [0, 0, 0]
])

Ixx = (1/12) * M * (Ly**2 + Lz**2)
Iyy = (1/12) * M * (Lx**2 + Lz**2)
Izz = (1/12) * M * (Lx**2 + Ly**2)
I_c = sp.Matrix([
    [Ixx, 0, 0],
    [0, Iyy, 0],
    [0, 0, Izz]
])

In [56]:
k_cEE = 0.5 * M4 * v_cEE.T * v_cEE + 0.5 * w_EE.T * I_cEE * w_EE
k_c4 = 0.5 * M2 * v_c4.T * v_c4 + 0.5 * w_4.T * I_c4 * w_4
k_c3 = 0.5 * M * v_c3.T * v_c3 + 0.5 * w_3.T * I_c3 * w_3

# Total Kinetic Energy:
K = k_cEE + k_c4 + k_c3

In [57]:
# We assume that the gravity act in negative y-direction of the base frame
g_vec = sp.Matrix([0, -g, 0])

# Potential energies:
u_EE = -M4 * g_vec.T * P_cEE
u_4 = -M2 * g_vec.T * P_c4
u_3 = -M * g_vec.T * P_c3

# Total Potential Energy:
U = u_EE + u_4 + u_3

In [58]:
# Potential Energy derivates
U_diff_thetaEE = U.diff(thetaEE)
U_diff_theta4 = U.diff(theta4)
U_diff_theta3 = U.diff(theta3)

# Kinetic Energy derivates
K_diff_thetaEE = K.diff(thetaEE)
K_diff_theta4 = K.diff(theta4)
K_diff_theta3 = K.diff(theta3)

# Kinetic Energy time derivates
# First differentiate wrt. the derivates of the thetas: theta_dots
K_diff_thetaEE_dot = K.diff(thetaEE_dot)
K_diff_theta4_dot = K.diff(theta4_dot)
K_diff_theta3_dot = K.diff(theta3_dot)
# Now differentiate wrt. time: t
K_diff_thetaEE_dot_diff_t = K_diff_thetaEE_dot.diff(t)
K_diff_theta4_dot_diff_t = K_diff_theta4_dot.diff(t)
K_diff_theta3_dot_diff_t = K_diff_theta3_dot.diff(t)


In [68]:
tau = sp.Matrix([K_diff_thetaEE_dot_diff_t - K_diff_thetaEE + U_diff_thetaEE
                 , K_diff_theta4_dot_diff_t - K_diff_theta4 + U_diff_theta4
                 , K_diff_theta3_dot_diff_t - K_diff_theta3 + U_diff_theta3])
tau = tau.applyfunc(sp.simplify)

In [69]:
display(Math(sp.latex(tau)))

<IPython.core.display.Math object>

In [ ]:
L2_Angles = pd.read_excel("L2_Angles.xlsx")
L2_time = L2_Angles['Time [s]']
L2_theta = L2_Angles['Theta [rad]']